## Load required packages

In [ ]:
# python basic packages
import numpy as np
import matplotlib.pyplot as plt
import random
from scipy import stats
import time
import pickle
import pandas as pd

import dimod
from collections import defaultdict
from itertools import product
from pyomo.environ import *

# DWave packages
from dwave.system import DWaveSampler, EmbeddingComposite, LeapHybridSampler
from dwave.system import LeapHybridCQMSampler
import dwave.inspector

# Carugno et al. (2022) packages to generate random Jobshop instances
from job_shop_experiment import JobShopExperiment

# our packages
import utilities

## Generate and save a random JJS instance

In [ ]:
number_of_jobs = 2
number_of_machine = 2
range_of_processing_times = [1, 1]
order_of_jobs = 'cycle' # options: 'cycle', 'rand', 'randEven'
maximum_makespan = 5

# Generate a basic JJS instance
JJS = JobShopExperiment ('data')
instance = JJS.get_problem(machines=number_of_machine,
                jobs=number_of_jobs, 
                ops=number_of_machine, 
                time=range_of_processing_times, 
                ordering=order_of_jobs)

# Convert Carugno basic instance to be ready for our coding
num_jobs = number_of_jobs
num_machines = number_of_machine + 1
num_time_slots = maximum_makespan

J = range(1, num_jobs+1)  # jobs
M = range(1, num_machines + 1)   # machines
T = range(1, num_time_slots+1)  # time slots

p = {} # (j, m): processing times
mp = {} # (j, m): previous machine
d = {} # j: due dates
w = {}  # j: tardiness weights
E = {}  # (j, m): earliness weights
f = {} # j: last job

for i, tasks in instance.items():
    tmpsum = 0
    job = int("".join([char for char in i if char.isnumeric()])) + 1
    tmp2 = 0
    processTime = 0
    perv = 0
    machine = 0

    for j in tasks:
        tmp2 = j[0]
        perv = machine
        machine = int("".join([char for char in tmp2 if char.isnumeric()])) + 1
        processTime = j[1]
        p[(job, machine)] = processTime
        mp[(job, machine)] = perv
        tmpsum = tmpsum + processTime
        randn = random.uniform(0, 0.5)
        E[(job, machine)] = round(randn, 3)
    f[job] = machine
    p[(job, num_machines)] = 0
    E[(job, num_machines)] = 0
    mp[(job, num_machines)] = machine
    d[job] = round(tmpsum*1.5)
    if job/num_jobs <= 0.20:
        w[job] = 4
    elif job/num_jobs <= 0.80:
        w[job] = 2
    else:
        w[job] = 1

h = [1, 1000, 1000, 1000, 1000, 1000]


# Variables to save
variables_to_save = {
    "num_jobs": num_jobs,
    "num_machines": num_machines,
    "num_time_slots": num_time_slots,
    "J": J,
    "M": M,
    "T": T,
    "p": p,
    "mp": mp,
    "d": d,
    "w": w,
    "E": E,
    "f": f,
    "h": h
}

# Save these variables to a file using pickle
instance_name = f'JJS_{num_jobs}_{num_machines}_{int(time.time())}.pkl'
with open('data/'+instance_name, 'wb') as file:
    pickle.dump(variables_to_save, file)


## Create the QUBO for the generated instance

In [ ]:
# load an instance
# instance_name = 'JJS_2_3_1723184886.pkl'
with open('data/'+instance_name, 'rb') as file:
    loaded_variables = pickle.load(file)

# Assign variables from the loaded dictionary
num_jobs = loaded_variables["num_jobs"]
num_machines = loaded_variables["num_machines"]
num_time_slots = loaded_variables["num_time_slots"]
J = loaded_variables["J"]
M = loaded_variables["M"]
T = loaded_variables["T"]
p = loaded_variables["p"]
mp = loaded_variables["mp"]
d = loaded_variables["d"]
w = loaded_variables["w"]
E = loaded_variables["E"]
f = loaded_variables["f"]
h = loaded_variables["h"]

log_data = {'instance name': [], 'jobs': [], 'machines': [],
            'iteration': [], 'response': [], 'Q exe time':[], 'LP exe time': [], 'sample': [],
            'makespan before LP': [], 'WET cost before LP': [], 'makespan after LP': [], 'WET cost after LP': [], 
            'num_time_slots': [], 'penalty': [], 'shots': []}

stopping_criteria = False
shots = 1000
iteration = 0
last_num_time_slots = 0
penalty = 1000
h = [1, penalty, penalty, penalty, penalty, penalty]

while last_num_time_slots != num_time_slots:
    iteration = iteration + 1
    
    start_time = time.time()
    qubo = utilities.generate_qubo_with_completion_time_variable(num_jobs, num_machines, num_time_slots, J, M, T, w, E, p, d, mp, h)
    response = utilities.solve_qubo_by_DWave_software_QA(qubo, shots)
    Qexetime = time.time() - start_time
    print('QA is completed at iteration: ', iteration)
    
    s_index = 0
    for sample, energy in response.data(['sample', 'energy']):
        s_index = s_index + 1
        finish_times = {}
        for variable, value in sample.items():
            if value == 1:
                finish_times[(variable[0], variable[1])] = variable[2]
        
        # print("Energy before LP: ", energy+qubo[(), ()])
        if energy+qubo[(), ()] < 100:
            start_time = time.time()
            model, results = utilities.solve_LP_after_QA(finish_times, num_jobs, num_machines, num_time_slots, J, M, T, w, E, p, d, mp, f)
            LPexetime = time.time() - start_time
            makespan = max(model.start_time[j, model.f[j]].value + model.t[j,model.f[j]] for j in model.J)
    
            log_data['instance name'].append(instance_name)
            log_data['jobs'].append(num_jobs)
            log_data['machines'].append(num_machines)
            log_data['iteration'].append(iteration)
            log_data['response'].append(response)
            log_data['Q exe time'].append(Qexetime)
            log_data['LP exe time'].append(LPexetime)
    
            log_data['sample'].append(s_index)
            log_data['makespan before LP'].append(max(finish_times.values()))
            log_data['WET cost before LP'].append(energy+qubo[(), ()])
            log_data['makespan after LP'].append(makespan)
            log_data['WET cost after LP'].append(model.obj())
            
            log_data['num_time_slots'].append(num_time_slots)
            log_data['penalty'].append(penalty)
            log_data['shots'].append(shots)
    
            print("Energy before LP: ", energy+qubo[(), ()], "  Energy after LP: ", model.obj())

    df = pd.DataFrame(log_data)
    data_for_hypo = df[(df['instance name'] == instance_name) & 
                 (df['iteration'] == iteration)]
    hypo_status, makespan = utilities.calculate_hypothesis_tests(data_for_hypo, alpha = 0.05)
    last_num_time_slots = num_time_slots
    num_time_slots = makespan
    T = range(1, num_time_slots+1)  # time slots
    penalty = penalty*0.8
    h = [1, penalty, penalty, penalty, penalty, penalty]

df.to_pickle('solutions/'+instance_name,)

In [ ]:
# timing_info = response.info['timing']
# # Logging the information
# print("Quantum Processing Unit (QPU) Time (in microseconds):", timing_info['qpu_access_time'])
# print("QPU Programming Time (in microseconds):", timing_info['qpu_programming_time'])
# print("Number of Shots:", shots)
# print("Samples and their energies:")

# # Loop through the samples
# for sample, energy in response.data(['sample', 'energy']):
#     print("Sample:", sample, "Energy:", energy)
# dwave.inspector.show(response)